In [1]:
import zarr
import numpy as np
import napari 
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial
import pickle
from nd2 import ND2File

Load nucleus dataset:

In [4]:
z = zarr.open('/path/to/experiment/images/nucleus_images_zarr/P00XX.zarr/0')
ch = 2
img = np.array(z[0,ch,::2,::2,::2])
n = napari.view_image(img)
l = n.add_labels(np.zeros_like(img))
#If retraining on image
#l = n.add_labels(l.data)

Run classifier on labeled images:

In [5]:
sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                    intensity=True, edges=False, texture=True,
                    sigma_min=sigma_min, sigma_max=sigma_max)
features = features_func(img)
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                            max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(l.data, features, clf)
result = future.predict_segmenter(features, clf)

pickle.dump(clf, open('/path/to/experiment/analysis/nucs_rf_model.pickle', 'wb'))

Check performance on classifier on other images:

In [6]:
z2 = zarr.open('/path/to/experiment/images/nucleus_images_zarr/P00XX.zarr/0')
img2 = np.array(z2[0,ch,::2,::2,::2])
features = features_func(img2)
result2 = future.predict_segmenter(features, clf)
n = napari.view_image(img2)
n.add_labels(result2)

<Labels layer 'result2' at 0x2cf80b35730>

Load spot image:

In [8]:
z = zarr.open('/path/to/experiment/images/sequential_images_zarr/P00XX.zarr/0')
spot_frame = 53
img = np.array(z[spot_frame,1,::2,::2,::2])
n = napari.view_image(img)
l = n.add_labels(np.zeros_like(img))
#If retraining on image
#l = n.add_labels(l.data)

Train classifier:

In [9]:
sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                    intensity=True, edges=True, texture=False,
                    sigma_min=sigma_min, sigma_max=sigma_max)
features = features_func(img)
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1,
                            max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(l.data, features, clf)
result = future.predict_segmenter(features, clf)
pickle.dump(clf, open('/path/to/experiment/analysis/spots_rf_model.pickle', 'wb'))

Validate results

In [12]:

sigma_min = 1
sigma_max = 16
features_func = partial(feature.multiscale_basic_features,
                    intensity=True, edges=True, texture=False,
                    sigma_min=sigma_min, sigma_max=sigma_max)
clf = pickle.load(open('/path/to/experiment/analysis/spots_rf_model.pickle', 'rb'))
z2 = zarr.open('/path/to/experiment/images/sequential_images_zarr/P00XX.zarr/0')
img2 = np.array(z2[spot_frame,1,::2,::2,::2])
features = features_func(img2)
result2 = future.predict_segmenter(features, clf)
n = napari.view_image(img2)
n.add_labels(result2)

<Labels layer 'result2' at 0x2cf8c309040>